In [1]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import numpy as np
import pandas as pd 
import pickle

c:\Users\chile\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Leer el archivo data/df_lyrics_clean.json
df_tracks = pd.read_json('data/df_lyrics_clean.json')
# Leer el archivo data/playlist.csv
df_playlist = pd.read_csv('data/playlist.csv')

In [3]:
# Función para crear los conjuntos de entrenamiento y prueba
def create_train_test_sets(df, test_fraction=0.2):
    # Listas para almacenar los conjuntos de datos de prueba y entrenamiento
    train_list = []
    test_list = []
    
    # Iterar sobre cada pid único
    for pid, group in df.groupby('pid'):
        # Seleccionar aleatoriamente el 20% de las canciones del grupo actual
        test_indices = group.sample(frac=test_fraction).index
        test_set = group.loc[test_indices]
        train_set = group.drop(test_indices)
        
        # Agregar los resultados a las listas
        train_list.append(train_set)
        test_list.append(test_set)
    
    # Concatenar todos los grupos de entrenamiento y prueba en dos DataFrames
    train_set = pd.concat(train_list).reset_index(drop=True)
    test_set = pd.concat(test_list).reset_index(drop=True)
    
    return train_set, test_set

# Crear los conjuntos de entrenamiento y prueba
train_data, test_data = create_train_test_sets(df_playlist)


In [6]:
import torch

# Verifica si hay una GPU disponible
if torch.cuda.is_available():
    device = torch.device("cuda")  # Usa GPU
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # Usa CPU si no hay GPU disponible
    print("GPU no disponible, usando CPU.")

GPU disponible: NVIDIA GeForce RTX 2060


In [7]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Verifica si hay una GPU disponible y configúrala como dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Cargar el modelo y tokenizer de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)  # Mueve el modelo a la GPU si está disponible

# Función para obtener embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length').to(device)  # Mueve los inputs a la GPU
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mueve los outputs de vuelta a la CPU para convertir a numpy

# Filtrar los track ids en el conjunto de entrenamiento
train_tids = train_data['tid'].unique()

# Crear un diccionario para almacenar los embeddings de entrenamiento
embeddings_dict_train = {}

# Generar embeddings solo para los tracks en el conjunto de entrenamiento
for tid in train_tids:
    track_info = df_tracks[df_tracks['tid'] == tid].iloc[0]
    content = f"{track_info['track_name']} {track_info['album_name']} {track_info['artist_name']} {track_info['lyrics']}"
    embeddings_dict_train[tid] = get_embeddings(content)

# Ahora los embeddings están almacenados en embeddings_dict_train y se han generado usando la GPU para acelerar el proceso

Using device: cuda


In [ ]:
# Guardar los embeddings junto con su track id
with open('data/embeddings_train.pkl', 'wb') as f:
    pickle.dump(embeddings_dict_train, f)